In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path

# Get the parent directory (i.e. project root)
project_root = Path().resolve().parent.parent 
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from functools import partial

from tqdm import tqdm

from tokenization.byte_pair_encoding.get_tokenizers import train_and_save_tokenizer_for, load_tokenizer_from

from pre_training.text_summarization.dataset import TextSummarizationDataset

from src.embedding import CustomEmbedding
from src.transformer import EncoderDecoderTransformer
from src.utils import padding_collate_fn

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/paul/opt/miniconda3/envs/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DF_DATA_PATH = '../../data/SAMSum/'

BPE_IN_PATH = '../../data/SAMSum/train_summary_and_dialogue.txt'
BPE_OUT_PATH = '../../tokenization/trained_tokenizers/SAMSum_BPE'

In [61]:
MAX_CONTEXT_WINDOW = 100

In [57]:
train_df = pd.read_json(DF_DATA_PATH + 'train_df.json', orient = 'records', lines = True)
val_df = pd.read_json(DF_DATA_PATH + 'val_df.json', orient = 'records', lines = True)
test_df = pd.read_json(DF_DATA_PATH + 'test_df.json', orient = 'records', lines = True)

In [6]:
bpe_tokenizer = train_and_save_tokenizer_for(in_file_paths = [BPE_IN_PATH], out_file_dir_path = BPE_OUT_PATH, vocab_size = 4_000)
pretrained_bpe_tokenizer = load_tokenizer_from(dir_path = BPE_OUT_PATH, model_max_length = 10000)

In [63]:
train_encoded = pretrained_bpe_tokenizer(
    train_df['dialogue'].tolist(),
    add_special_tokens = False
)

train_encoded_filtered = [example for example in train_encoded.data['input_ids'] if len(example) <= MAX_CONTEXT_WINDOW]

print(len(train_encoded.data['input_ids']))
print(len(train_encoded_filtered))

14732
5580


In [43]:
print(type(train_encoded))
print(type(train_encoded[0]))

<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'tokenizers.Encoding'>


In [60]:
print(train_encoded.data.keys())
print(len(train_encoded.data['input_ids']))
print(train_encoded.data['input_ids'][0])
print(train_encoded.encodings[0].tokens)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
14732
[1607, 30, 273, 274, 598, 307, 225, 3972, 18, 822, 286, 449, 441, 35, 206, 203, 2115, 30, 893, 5, 206, 203, 1607, 30, 273, 419, 933, 286, 606, 1677]
['Amanda', ':', 'ĠI', 'Ġb', 'ak', 'ed', 'Ġ', 'Ġcookies', '.', 'ĠDo', 'Ġyou', 'Ġwant', 'Ġsome', '?', 'č', 'Ċ', 'Jerry', ':', 'ĠSure', '!', 'č', 'Ċ', 'Amanda', ':', 'ĠI', "'ll", 'Ġbring', 'Ġyou', 'Ġtomorrow', 'Ġ:-)']


In [47]:
print(train_encoded[14731].tokens)
print(train_encoded[0])

['Georg', 'ia', ':', 'Ġare', 'Ġyou', 'Ġready', 'Ġfor', 'Ġhotel', 'Ġhun', 'ting', '?', 'ĠWe', 'Ġneed', 'Ġto', 'Ġbook', 'Ġsomething', 'Ġfinally', 'Ġfor', 'ĠL', 'is', 'b', 'on', 'č', 'Ċ', 'Jul', 'i', 'ette', ':', 'Ġsure', 'Ġwe', 'Ġcan', 'Ġgo', 'Ġon', ',', 'Ġshow', 'Ġme', 'Ġwhat', 'Ġyou', 'Ġfound', 'č', 'Ċ', 'Georg', 'ia', ':', 'Ġ<', 'file', '_', 'photo', '>', 'č', 'Ċ', 'Jul', 'i', 'ette', ':', 'Ġn', 'ah', '...', 'Ġit', 'Ġlooks', 'Ġlike', 'Ġan', 'Ġold', 'Ġlady', "'s", 'Ġroom', 'Ġlol', 'č', 'Ċ', 'Georg', 'ia', ':', 'Ġ<', 'file', '_', 'photo', '>', 'č', 'Ċ', 'Jul', 'i', 'ette', ':', 'Ġthat', "'s", 'Ġbetter', '...', 'Ġbut', 'Ġthe', 'Ġbed', 'Ġdoesn', "'t", 'Ġlook', 'Ġvery', 'Ġcomfort', 'able', 'č', 'Ċ', 'Georg', 'ia', ':', 'Ġi', 'Ġkind', 'Ġof', 'Ġlike', 'Ġit', 'Ġand', 'Ġit', "'s", 'Ġreally', 'Ġclose', 'Ġto', 'Ġthe', 'Ġcity', 'Ġc', 'en', 'ter', 'č', 'Ċ', 'Jul', 'i', 'ette', ':', 'Ġshow', 'Ġme', 'Ġthe', 'Ġothers', 'Ġplease', 'č', 'Ċ', 'Georg', 'ia', ':', 'Ġ<', 'file', '_', 'photo', '>', 'č', 'Ċ'